# Power BI / Fabric Semantic Model Connection (Local)

This notebook demonstrates how to connect to Power BI Semantic Models (formerly Datasets) using the `semantic-link` (sempy) library from a local environment.

## Prerequisites
1. Ensure you have authenticated with Azure. The library successfully uses credentials from the Azure CLI (`az login`) or prompts for interactive authentication.
2. You need access to the workspace and the semantic models.
3. XMLA Read access might be required for some operations.

In [ ]:
import sempy.fabric as fabric
import pandas as pd

# # Verify installation and print version
# import importlib.metadata
# print(f"Semantic Link version: {importlib.metadata.version('semantic-link')}")

In [ ]:
import sys
from azure.identity import DeviceCodeCredential, DefaultAzureCredential
from azure.core.exceptions import ClientAuthenticationError

# Attempt to authenticate
try:
    print("Trying DefaultAzureCredential...")
    credential = DefaultAzureCredential()
    # Test the credential by getting a token for Power BI
    token = credential.get_token("https://analysis.windows.net/powerbi/api/.default")
    print("Successfully authenticated using DefaultAzureCredential.")
except ClientAuthenticationError:
    print("DefaultAzureCredential failed. Falling back to DeviceCodeCredential.")
    print("Please follow the instructions below to authenticate:")
    credential = DeviceCodeCredential()
    token = credential.get_token("https://analysis.windows.net/powerbi/api/.default")
    print("\nSuccessfully authenticated using DeviceCodeCredential.")
except Exception as e:
    print(f"Authentication failed: {e}") 
    
# NOTE: SemPy usually picks up the context automatically if you are logged in via AZ CLI.
# If you used DeviceCodeCredential above, your local environment should now have a cached token 
# or the browser session is active. However, 'sempy' primarily looks for AZ CLI or Environment vars.
# If the above passes but sempy still fails, please run '!az login' in a new cell.

In [ ]:
import json
import base64

def get_user_from_token(token_obj):
    try:
        # JWT is header.payload.signature
        # We need the payload (index 1)
        token_str = token_obj.token
        payload_part = token_str.split('.')[1]
        
        # Base64 decode (add padding if needed)
        padding = '=' * (4 - len(payload_part) % 4)
        payload_decoded = base64.urlsafe_b64decode(payload_part + padding).decode('utf-8')
        claims = json.loads(payload_decoded)
        
        # Common claims for identity
        user = claims.get('upn') or claims.get('unique_name') or claims.get('email') or "Unknown"
        name = claims.get('name', 'Unknown')
        
        print(f"Logged in User: {user}")
        print(f"Name: {name}")
        print(f"Tenant ID: {claims.get('tid')}")
        return user
    except Exception as e:
        print(f"Could not decode token to find user: {e}")
        return None

if 'token' in locals():
    print("Decoding current token...")
    get_user_from_token(token)
else:
    print("No token variable found. Run the authentication cell above.")

# Check Azure CLI context from Python as well
import subprocess
try:
    print("\n--- Azure CLI 'az account show' ---")
    result = subprocess.run(["az", "account", "show"], capture_output=True, text=True)
    if result.returncode == 0:
        account_info = json.loads(result.stdout)
        print(f"CLI User: {account_info.get('user', {}).get('name')}")
        print(f"CLI Environment: {account_info.get('environmentName')}")
    else:
        print("Azure CLI not logged in or not found.")
except Exception as e:
    print(f"Error checking CLI: {e}")

## 1. List Workspaces
First, let's verify we can connect by listing accessible workspaces.

In [ ]:
# List all workspaces you have access to
# We pass the 'credential' object explicitly to avoid FabricAnalyticsTokenCredentialProvider errors
try:
    df_workspaces = fabric.list_workspaces(endpoint="powerbi", credential=credential)
    display(df_workspaces.head())
except NameError:
    print("The 'credential' variable is not defined. Please run the authentication cell above.")
except Exception as e:
    print(f"Error listing workspaces: {e}")

## 2. List Semantic Models (Datasets)
Specify a workspace name (optional if you want to look in the 'current' workspace context, but for local execution, explicit workspace is usually safer).

Replace `"YOUR_WORKSPACE_NAME"` with your actual workspace name.

In [ ]:
workspace_name = "perry-pbi-demo-workspace"

# List datasets in the specific workspace
try:
    df_datasets = fabric.list_datasets(workspace=workspace_name, credential=credential)
    display(df_datasets)
except Exception as e:
    print(f"Error listing datasets: {e}")

## 3. Read Data from a Table
Once you have the dataset name, you can list tables and read data.
Replace `"YOUR_DATASET_NAME"` with the name of a dataset found above.

In [ ]:
dataset_name = "Palantir API Demo - Azure Gov - AUSA 2025"

# List tables in the dataset
try:
    # Pass credential to list_tables
    df_tables = fabric.list_tables(dataset=dataset_name, workspace=workspace_name, credential=credential)
    display(df_tables.head())
    
    # Pick the first table name to read, or specify one manually
    if not df_tables.empty:
        table_name = df_tables.iloc[0]['Name']
        print(f"Reading table: {table_name}")
        
        # Pass credential to read_table
        df_data = fabric.read_table(dataset=dataset_name, table=table_name, workspace=workspace_name, credential=credential)
        display(df_data.head())
except Exception as e:
    print(f"Error reading table: {e}")

## 4. Evaluate DAX Measure
You can also run DAX queries or evaluate specific measures directly.

In [ ]:
# Example: Evaluate a simple measure (or check what measures exist first)
try:
    # Pass credential to list_measures
    df_measures = fabric.list_measures(dataset=dataset_name, workspace=workspace_name, credential=credential)
    display(df_measures.head())
    
    if not df_measures.empty:
        measure_name = df_measures.iloc[0]['Name']
        print(f"Evaluating measure: {measure_name}")
        
        # Evaluate measure
        # Note: You can add 'groupby_columns' list as the 3rd argument if needed
        # Pass credential to evaluate_measure
        result = fabric.evaluate_measure(dataset=dataset_name, measure=measure_name, workspace=workspace_name, credential=credential)
        display(result)
except Exception as e:
    print(f"Error evaluating measure: {e}")